In [75]:
import time
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from IPython.display import display, HTML
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

In [76]:
dong = '화정동'
url = "https://new.land.naver.com/offices?ms=35.1433487,126.8801591,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"
gcs = url[url.find("3"):url.find(",16")]

In [ ]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
driver.get(url)
driver.execute_script("window.open('https://www.movable-type.co.uk/scripts/latlong-utm-mgrs.html');")

In [ ]:
driver.switch_to.window(driver.window_handles[1])
driver.find_element(By.CSS_SELECTOR, "input#deg-format-d").click()
driver.find_element(By.CSS_SELECTOR, "input#precision-mm").click()

In [ ]:
gcs_input = driver.find_element(By.CSS_SELECTOR, "input#latlon")
gcs_input.clear()
gcs_input.send_keys(gcs)
gcs_input.send_keys(Keys.RETURN)
time.sleep(0.1)
utm_output = driver.find_element(By.CSS_SELECTOR, "input#utm-coord").get_attribute("value").split(" ")

In [ ]:
driver.switch_to.window(driver.window_handles[0])
time.sleep(3)

columns = ['동', '위도', '경도', '상가종류', '월세', '평수', '층수']
df = pd.DataFrame(columns=columns)

pin = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")

In [ ]:
driver.refresh()

In [ ]:
for i in range(len(pin)):
# for i in range(2):
    while 1:
        map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
        my_x = map_element.size['width'] / 2 + driver.find_element(By.CSS_SELECTOR, "div#listContents1").size['width']/2
        # my_x = map_element.size['width'] / 2
        my_y = map_element.size['height'] / 2

        time.sleep(1)
        pin = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
        target_x = float(pin[i].get_attribute("style").split(" ")[-1].replace("px;", ""))
        target_y = float(pin[i].get_attribute("style").split(" ")[-3].replace("px;", ""))
        if abs(target_x) > 10000 or abs(target_y) > 10000:
            driver.refresh()
            time.sleep(3)
        else :
            break

    if target_x < 380 or target_y < -30:
        continue

    gap_x = target_x - my_x
    gap_y = target_y - my_y

    distance_x = gap_x * 100 / 52
    distance_y = gap_y * 100 / 52

    utm_x = float(utm_output[2]) + distance_x
    utm_y = float(utm_output[3]) - distance_y
    
    print(i+1)
    print(f"타깃 좌표 : x : {target_x}, y : {target_y}")
    print("52 N",utm_x, utm_y)

    pin[i].click()

    try:
        body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
        while True:
            body.send_keys(Keys.END)

            time.sleep(1)
            loader = driver.find_element(By.CSS_SELECTOR, 'div.loader')
    except:
        pass
    
    data = driver.find_elements(By.CSS_SELECTOR, "div.item")

    for j in range(len(data)):
        text_list = data[j].text.strip().split('\n')

        idx = 1 if text_list[0] == "공인중개사협회매물" else 0  # 조건에 따라 인덱스 설정

        split_values = text_list[idx + 2].split(', ')  # 공통적으로 필요한 값

        df.loc[i] = [
            dong,
            utm_x,
            utm_y,
            text_list[idx],
            text_list[idx + 1],
            split_values[0],
            split_values[1]
        ]

    time.sleep(2)
    pin[i].click()



time.sleep(1)
driver.switch_to.window(driver.window_handles[1])

for k in range(df.shape[0]):
    # UTM 좌표 입력 필드 가져오기
    utm_input = driver.find_element(By.CSS_SELECTOR, "input#utm-coord")

    # 기존 값 삭제 후 새로운 UTM 좌표 입력
    utm_input.clear()
    utm_input.send_keys("52 N " + str(df.loc[k, '위도']) + ' ' + str(df.loc[k, '경도']))
    utm_input.send_keys(Keys.RETURN)
    time.sleep(0.1)

    # 변환된 위도/경도 가져오기
    latlon_value = driver.find_element(By.CSS_SELECTOR, "input#latlon").get_attribute("value")

    latlon_value.find(", ")
    lat, long = latlon_value.split(", ")
    df.loc[k, '위도'] = float(lat[:-3])
    df.loc[k, '경도'] = float(long[:-3])

df.to_csv(f"./data/03_부동산/{dong}_부동산.csv", encoding='utf-8-sig', index=False)
df[['위도', '경도']].to_csv(f"./data/04_부동산_test/{dong}_test.csv", encoding='utf-8-sig', index=False, header=False)
driver.switch_to.window(driver.window_handles[0])



In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
줌 18로 파란 핑을 가운데 좌표에 놓고 그 곳에 url가져와서 좌표 저장하기
그 핑의 데이터들의 매물번호, 월세, 평수, m2, 층수, 추천업종, 매물특징 가져오기

In [ ]:
import re
import time
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from IPython.display import display, HTML
from selenium.webdriver.common.by import By
from pyproj import Proj, transform, Transformer
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
dong_name = ['광천동', '금호동', '내방동', '농성동', '덕흥동', '동천동', '마륵동', '매월동', '벽진동', '서창동', '세하동', '쌍촌동', '양동', '용두동', '유촌동', '치평동', '풍암동', '화정동']
url = "https://new.land.naver.com/offices?ms=35.1433487,126.8801591,18&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"

In [ ]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
driver.get(url)

In [ ]:
import json
import numpy as np

# GeoJSON 불러오기
with open('./test.geojson', 'r', encoding='utf-8') as f:
    geojson = json.load(f)

coordinates = geojson['features'][0]['geometry']['coordinates'][0]

# 위도, 경도 각각 분리
lons, lats = zip(*coordinates)
min_lon, max_lon = min(lons), max(lons)
min_lat, max_lat = min(lats), max(lats)

# 간격 설정 (지도 하나에 커버되는 영역 기준, 0.004 정도가 적당)
lat_step = 0.004
lon_step = 0.004

# 그리드 생성
grid_points = []
lat = min_lat
while lat <= max_lat:
    lon = min_lon
    while lon <= max_lon:
        grid_points.append((round(lat, 7), round(lon, 7)))
        lon += lon_step
    lat += lat_step

# 생성된 포인트 확인
for point in grid_points:
    print(f"https://new.land.naver.com/offices?ms={point[0]},{point[1]},18")


In [ ]:
from selenium import webdriver
import time

# ChromeDriver 경로 설정
driver = webdriver.Chrome()

for lat, lon in grid_points:
    url = f"https://new.land.naver.com/offices?ms={lat},{lon},18&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"
    driver.get(url)
    
    time.sleep(0.5)  # 페이지 로딩 대기 (필요시 더 늘리기)

    # 스크롤 등 추가 동작도 가능
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# 완료 후 브라우저 종료
driver.quit()


In [ ]:
len(grid_points)
# grid_points

In [ ]:
df = pd.DataFrame(grid_points, columns=['y', 'x'])

# CSV로 저장 (인코딩: utf-8-sig, 인덱스/헤더 없이 저장)
df.to_csv('grid_points.csv', encoding='utf-8-sig', index=False, header=False)

In [4]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyproj import Proj, Transformer

# -------------------- 기본 설정 --------------------
driver = webdriver.Chrome()
df = pd.DataFrame(columns=['동', 'utm_x', 'utm_y', '타입', '가격', '면적', '층'])
dong = "예시동"

# 거리 및 좌표 변환 설정
m, px = 30, 61  # 예시 값
meter_per_px = m / px

zone_number = 52
northern = True

# 좌표 변환기 설정
utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=not northern)
wgs84_proj = Proj(proj='latlong', datum='WGS84')

# 양방향 변환기
transformer_to_wgs = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)
transformer_to_utm = Transformer.from_proj(wgs84_proj, utm_proj, always_xy=True)
transformer = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)

origin_x, origin_y = transformer_to_utm.transform(lon, lat)
print(f"원점 UTM 좌표 설정: {origin_x}, {origin_y}")

# -------------------- 좌표 로드 --------------------
latlon_df = pd.read_csv('./files/seogu_filt.csv', header=None, names=['lat', 'lon'])

cnt = 1
for idx, row in latlon_df.iterrows():
    cnt += 1
    if cnt > 10:
        break
    lat, lon = row['lat'], row['lon']

    # 위도/경도 → UTM 변환: 원점 설정
    origin_x, origin_y = transformer_to_utm.transform(lon, lat)
    print(f"원점 UTM 좌표 설정: {origin_x}, {origin_y}")

    print(f"[{idx+1}] 이동: {lat}, {lon}")
    url = f"https://new.land.naver.com/offices?ms={lat},{lon},18&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"
    driver.get(url)
    driver.maximize_window()
    time.sleep(2)

    # 핀 요소 찾기
    try:
        pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
    except:
        print("핀 로딩 실패")
        continue

    for i in range(len(pin_elements)):
        while True:
            try:
                map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
                my_x = map_element.size['width'] / 2 + driver.find_element(By.CSS_SELECTOR, "div#listContents1").size['width']/2
                my_y = map_element.size['height'] / 2

                pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
                target_x = float(pin_elements[i].get_attribute("style").split(" ")[-1].replace("px;", ""))
                target_y = float(pin_elements[i].get_attribute("style").split(" ")[-3].replace("px;", ""))

                if abs(target_x) > 10000 or abs(target_y) > 10000:
                    driver.refresh()
                    time.sleep(3)
                    continue
                else:
                    break
            except:
                time.sleep(1)

        if target_x < 380 or target_y < -30:
            continue

        gap_x = target_x - my_x
        gap_y = target_y - my_y

        distance_x = gap_x * meter_per_px
        distance_y = gap_y * meter_per_px

        utm_x = origin_x + distance_x
        utm_y = origin_y - distance_y

        print(f"타깃 좌표 : x : {target_x}, y : {target_y}")
        print("52 N", utm_x, utm_y)

        try:
            pin_elements[i].click()
        except:
            pass
        time.sleep(1)

        try:
            body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
            while True:
                body.send_keys(Keys.END)
                time.sleep(1)
                driver.find_element(By.CSS_SELECTOR, 'div.loader')
        except:
            pass

        items = driver.find_elements(By.CSS_SELECTOR, "div.item")

        for j in range(len(items)):
            text_list = items[j].text.strip().split('\n')
            idx_offset = 1 if text_list[0] == "공인중개사협회매물" else 0
            split_values = text_list[idx_offset + 2].split(', ')

            df.loc[len(df)] = [
                dong,
                utm_x,
                utm_y,
                text_list[idx_offset],
                text_list[idx_offset + 1],
                split_values[0],
                split_values[1]
            ]

        try:
            pin_elements[i].click()
        except:
            pass
        
        time.sleep(2)

# -------------------- UTM → 위도/경도 변환 --------------------
utm_coords = list(zip(df['utm_x'], df['utm_y']))
latlon_list = [transformer.transform(x, y) for x, y in utm_coords]

df['경도'] = [round(lon, 7) for lon, lat in latlon_list]
df['위도'] = [round(lat, 7) for lon, lat in latlon_list]

# -------------------- CSV 저장 --------------------
df.to_csv(f"./data/03_부동산/{dong}_부동산.csv", encoding='utf-8-sig', index=False)
df[['위도', '경도']].to_csv(f"./data/04_부동산_test/{dong}_test.csv", encoding='utf-8-sig', index=False, header=False)

print("CSV 저장 완료")


원점 UTM 좌표 설정: 305016.9112861915, 3887884.6064145244
원점 UTM 좌표 설정: 302100.2437171852, 3887947.755934084
[1] 이동: 35.1149426, 126.8284109
타깃 좌표 : x : 1519.29, y : 54.4778
52 N 302276.94371718523 3888102.193081625
원점 UTM 좌표 설정: 302464.82830759406, 3887939.8109026547
[2] 이동: 35.1149426, 126.8324109
타깃 좌표 : x : 1519.29, y : 54.4778
52 N 302641.52830759407 3888094.2480501956
원점 UTM 좌표 설정: 302829.41256985866, 3887931.880540455
[3] 이동: 35.1149426, 126.8364109
타깃 좌표 : x : 773.64, y : 54.4778
52 N 302639.39945510455 3888086.317687996
원점 UTM 좌표 설정: 303193.99650457845, 3887923.9648473677
[4] 이동: 35.1149426, 126.8404109
원점 UTM 좌표 설정: 303558.58011236234, 3887916.063823277
[5] 이동: 35.1149426, 126.8444109
타깃 좌표 : x : 2451.17, y : 467.871
52 N 304193.5817517066 3887867.1928396705
원점 UTM 좌표 설정: 303923.16339380975, 3887908.1774680633
[6] 이동: 35.1149426, 126.8484109
타깃 좌표 : x : 1705.52, y : 467.871
52 N 304191.4519183999 3887859.3064844566
원점 UTM 좌표 설정: 304287.74634952727, 3887900.305781612
[7] 이동: 35.1149